# Automate Read Across

## Choose Alerts

In [1]:
import BB_calculator
import Benigni_Bossa_index as FAI

## Scegli alerts in function vera

In [2]:
# rdkit packages
from rdkit import Chem
from rdkit.Chem import Fragments
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
IPythonConsole.ipython_useSVG=True 
from rdkit.Chem.Draw import SimilarityMaps
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem.Draw import rdMolDraw2D

# Data Analisys library
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np

# standard library
import subprocess
import os
from tqdm import tqdm

# library useful for output
from fpdf import FPDF
from PyPDF2 import PdfFileMerger
import xlsxwriter
from tqdm import tqdm
import matplotlib.pyplot as plt
from rdkit.Chem.Draw import rdMolDraw2D
from fpdf import FPDF
from PyPDF2 import PdfFileMerger
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import ColorConverter 

# personal library and functions
from Function_Vera import*
import descriptor_calculator as dc
import Function_for_descriptor as fd
import Function_for_descriptor_tuple as Fdt
import Function_descriptor_idx as FDT_indx

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


### Define class for thread, useful for matrix calculation

In [3]:
import threading
class myThread (threading.Thread):
   
    def __init__(self, threadID, name, Dataset, df_input, path_program):
        threading.Thread.__init__(self)
        self.threadID = threadID
        self.name = name
        self.dataset = Dataset
        self.input = df_input
        self.path_program = path_program

    def make_matrix(self):        
        self.dataset['SMILES'].to_csv('dataset1.csv',header = None,index=False)
        self.input['SMILES'].to_csv('dataset2.csv',header = None,index=False)
        command = ["java", "-jar", "-Xmx8000m", self.path_program, "-similarity_ds", "dataset1.csv","dataset2.csv"]
        subprocess.call(command)
        df = pd.read_table('similarity_dataset1.csv_dataset2.csv.csv').drop('#', axis=1)
        os.remove('similarity_dataset1.csv_dataset2.csv.csv')
        os.remove('dataset1.csv')
        os.remove('dataset2.csv')
        df.rename(columns={'Mol':'SMILES'}, inplace=True)
        a = pd.merge(df, self.dataset, how='inner', on='SMILES')
        df = a.copy()
        return df

    def run(self):
        print ("Starting " + self.name)
        global df
        df = myThread.make_matrix(self)
        print ("Exiting " + self.name)

## Importa:

1. __Dataset:__ with header column: __'SMILES'__ and __'Experimental value'__ (note: keys sensitive and Active=1, inactive=0)

In [6]:
Dataset_path = 'dataset.xlsx'
Dataset = pd.read_excel(Dataset_path)

2. __Target:__ with header column: __'SMILES'__ and __'Experimental value'__


In [7]:
path_target = 'test_carcino3.xlsx'

3. __MatrixSimilarity:__ path of matrix similarity calculator program(CustumFeature)

In [8]:
path_program = "CustomFeaturesCLI/CustomFeaturesCLI.jar"

__Datset clean__

In [9]:
Dataset['SMILES'] = Dataset['SMILES'].apply(lambda x: x.strip())
Dataset.drop_duplicates(inplace=True)
Dataset = DataCuration(Dataset)
error = Dataset[1]
Dataset = Dataset[0]

[19:01:13] Can't kekulize mol.  Unkekulized atoms: 1 3 5 6 7 8 11
[19:01:13] Can't kekulize mol.  Unkekulized atoms: 1 3 5 6 7 8 11
[19:01:13] Can't kekulize mol.  Unkekulized atoms: 1 3 5 6 7 8 11
[19:01:13] Can't kekulize mol.  Unkekulized atoms: 1 3 5 6 7 8 11
[19:01:13] Can't kekulize mol.  Unkekulized atoms: 1 3 5 6 7 8 11
[19:01:13] Can't kekulize mol.  Unkekulized atoms: 1 3 5 6 7 8 11
[19:01:13] Can't kekulize mol.  Unkekulized atoms: 1 3 5 6 7 8 11
[19:01:13] Can't kekulize mol.  Unkekulized atoms: 1 3 5 6 7 8 11


__Target clean__

In [10]:
df_input = pd.read_excel(path_target)
if 'SMILES' not in df_input.keys():
    df_input = pd.read_excel(path_target, header=None) 
    df_input.rename(columns={0:'SMILES'}, inplace =True)
df_input = DataCurationTarget(df_input)
error = df_input[1]
df_input = df_input[0]

## MATRIX SIMILARITY

In [11]:
thread1 = myThread(1, "Thread-1", Dataset, df_input, path_program)
thread1.start()
thread1.join()
if 'SMILES_clean' in df.keys(): del df['SMILES_clean']
if 'SMILES_Can' in df.keys(): del df['SMILES_Can']
if 'SMILES_old' in df.keys(): del df['SMILES_old'] # 7/04 
if 'Unnamed: 0' in df.keys(): del df['Unnamed: 0']

Starting Thread-1
* Starting calculation...
* Calculating Vega Similarity between #1 :CC=O and COc1ccc(O)c(C(C)(C)C)c1...done!
* Calculating Vega Similarity between #1 :CC=O and C=CCc1ccc(OC)cc1...done!
* Calculating Vega Similarity between #2 :CC=NO and COc1ccc(O)c(C(C)(C)C)c1...done!
* Calculating Vega Similarity between #2 :CC=NO and C=CCc1ccc(OC)cc1...done!
* Calculating Vega Similarity between #3 :CC(N)=O and COc1ccc(O)c(C(C)(C)C)c1...done!
* Calculating Vega Similarity between #3 :CC(N)=O and C=CCc1ccc(OC)cc1...done!
* Calculating Vega Similarity between #4 :CC(=O)c1ccc(S(=O)(=O)NC(=O)NC2CCCCC2)cc1 and COc1ccc(O)c(C(C)(C)C)c1...done!
* Calculating Vega Similarity between #4 :CC(=O)c1ccc(S(=O)(=O)NC(=O)NC2CCCCC2)cc1 and C=CCc1ccc(OC)cc1...done!
* Calculating Vega Similarity between #5 :CC(C)=NNc1ncc(-c2ccc([N+](=O)[O-])o2)s1 and COc1ccc(O)c(C(C)(C)C)c1...done!
* Calculating Vega Similarity between #5 :CC(C)=NNc1ncc(-c2ccc([N+](=O)[O-])o2)s1 and C=CCc1ccc(OC)cc1...done!
* Calculati

* Calculating Vega Similarity between #45 :NCCCCCC(=O)O and C=CCc1ccc(OC)cc1...done!
* Calculating Vega Similarity between #46 :NCC1(CC(=O)O)CCCCC1 and COc1ccc(O)c(C(C)(C)C)c1...done!
* Calculating Vega Similarity between #46 :NCC1(CC(=O)O)CCCCC1 and C=CCc1ccc(OC)cc1...done!
* Calculating Vega Similarity between #47 :Nc1ncn[nH]1 and COc1ccc(O)c(C(C)(C)C)c1...done!
* Calculating Vega Similarity between #47 :Nc1ncn[nH]1 and C=CCc1ccc(OC)cc1...done!
* Calculating Vega Similarity between #48 :NCCCCCCCCCCC(=O)O and COc1ccc(O)c(C(C)(C)C)c1...done!
* Calculating Vega Similarity between #48 :NCCCCCCCCCCC(=O)O and C=CCc1ccc(OC)cc1...done!
* Calculating Vega Similarity between #49 :CCC1(CCC(C)C)C(=O)NC(=O)NC1=O and COc1ccc(O)c(C(C)(C)C)c1...done!
* Calculating Vega Similarity between #49 :CCC1(CCC(C)C)C(=O)NC(=O)NC1=O and C=CCc1ccc(OC)cc1...done!
* Calculating Vega Similarity between #50 :CCCCCN(N=O)C(N)=O and COc1ccc(O)c(C(C)(C)C)c1...done!
* Calculating Vega Similarity between #50 :CCCCCN(N=O)

* Calculating Vega Similarity between #91 :Nc1ccc(N)c2c1C(=O)c1c(N)ccc(N)c1C2=O and C=CCc1ccc(OC)cc1...done!
* Calculating Vega Similarity between #92 :CNc1ccc(N(CCO)CCO)cc1[N+](=O)[O-] and COc1ccc(O)c(C(C)(C)C)c1...done!
* Calculating Vega Similarity between #92 :CNc1ccc(N(CCO)CCO)cc1[N+](=O)[O-] and C=CCc1ccc(OC)cc1...done!
* Calculating Vega Similarity between #93 :O=[N+]([O-])c1cc(N(CCO)CCO)ccc1NCCO and COc1ccc(O)c(C(C)(C)C)c1...done!
* Calculating Vega Similarity between #93 :O=[N+]([O-])c1cc(N(CCO)CCO)ccc1NCCO and C=CCc1ccc(OC)cc1...done!
* Calculating Vega Similarity between #94 :ClC(Cl)Br and COc1ccc(O)c(C(C)(C)C)c1...done!
* Calculating Vega Similarity between #94 :ClC(Cl)Br and C=CCc1ccc(OC)cc1...done!
* Calculating Vega Similarity between #95 :CCBr and COc1ccc(O)c(C(C)(C)C)c1...done!
* Calculating Vega Similarity between #95 :CCBr and C=CCc1ccc(OC)cc1...done!
* Calculating Vega Similarity between #96 :CCCC1OC2CC3C4CCC5=CC(=O)C=CC5(C)C4C(O)CC3(C)C2(C(=O)CO)O1 and COc1ccc(O)c(

* Calculating Vega Similarity between #140 :FC(F)Cl and COc1ccc(O)c(C(C)(C)C)c1...done!
* Calculating Vega Similarity between #140 :FC(F)Cl and C=CCc1ccc(OC)cc1...done!
* Calculating Vega Similarity between #141 :FCCl and COc1ccc(O)c(C(C)(C)C)c1...done!
* Calculating Vega Similarity between #141 :FCCl and C=CCc1ccc(OC)cc1...done!
* Calculating Vega Similarity between #142 :COCCl and COc1ccc(O)c(C(C)(C)C)c1...done!
* Calculating Vega Similarity between #142 :COCCl and C=CCc1ccc(OC)cc1...done!
* Calculating Vega Similarity between #143 :CN(C)C(=O)Nc1ccc(Cl)cc1 and COc1ccc(O)c(C(C)(C)C)c1...done!
* Calculating Vega Similarity between #143 :CN(C)C(=O)Nc1ccc(Cl)cc1 and C=CCc1ccc(OC)cc1...done!
* Calculating Vega Similarity between #144 :Cc1nnc2n1-c1sc(CCc3ccc(CC(C)C)cc3)cc1C(c1ccccc1Cl)=NC2C and COc1ccc(O)c(C(C)(C)C)c1...done!
* Calculating Vega Similarity between #144 :Cc1nnc2n1-c1sc(CCc3ccc(CC(C)C)cc3)cc1C(c1ccccc1Cl)=NC2C and C=CCc1ccc(OC)cc1...done!
* Calculating Vega Similarity between

* Calculating Vega Similarity between #190 :OC(CBr)C(O)C(O)C(O)CBr and C=CCc1ccc(OC)cc1...done!
* Calculating Vega Similarity between #191 :BrCCBr and COc1ccc(O)c(C(C)(C)C)c1...done!
* Calculating Vega Similarity between #191 :BrCCBr and C=CCc1ccc(OC)cc1...done!
* Calculating Vega Similarity between #192 :Brc1cc(Br)c2cccnc2c1 and COc1ccc(O)c(C(C)(C)C)c1...done!
* Calculating Vega Similarity between #192 :Brc1cc(Br)c2cccnc2c1 and C=CCc1ccc(OC)cc1...done!
* Calculating Vega Similarity between #193 :CCCCNC(=O)N(CCCC)N=O and COc1ccc(O)c(C(C)(C)C)c1...done!
* Calculating Vega Similarity between #193 :CCCCNC(=O)N(CCCC)N=O and C=CCc1ccc(OC)cc1...done!
* Calculating Vega Similarity between #194 :CC(F)(Cl)Cl and COc1ccc(O)c(C(C)(C)C)c1...done!
* Calculating Vega Similarity between #194 :CC(F)(Cl)Cl and C=CCc1ccc(OC)cc1...done!
* Calculating Vega Similarity between #195 :O=C(O)C(Cl)Cl and COc1ccc(O)c(C(C)(C)C)c1...done!
* Calculating Vega Similarity between #195 :O=C(O)C(Cl)Cl and C=CCc1ccc(OC)c

...

* Calculating Vega Similarity between #1513 :O=[N+]([O-])OCC(CO[N+](=O)[O-])(CO[N+](=O)[O-])CO[N+](=O)[O-] and C=CCc1ccc(OC)cc1...done!
* Calculating Vega Similarity between #1514 :CCCCN(N=O)C1CCC(=O)O1 and COc1ccc(O)c(C(C)(C)C)c1...done!
* Calculating Vega Similarity between #1514 :CCCCN(N=O)C1CCC(=O)O1 and C=CCc1ccc(OC)cc1...done!
* Calculating Vega Similarity between #1515 :CCCCOP(=O)(OCCCC)OCCCC and COc1ccc(O)c(C(C)(C)C)c1...done!
* Calculating Vega Similarity between #1515 :CCCCOP(=O)(OCCCC)OCCCC and C=CCc1ccc(OC)cc1...done!
* Calculating Vega Similarity between #1516 :OCC1OC(OC2(CCl)OC(CCl)C(O)C2O)C(O)C(O)C1Cl and COc1ccc(O)c(C(C)(C)C)c1...done!
* Calculating Vega Similarity between #1516 :OCC1OC(OC2(CCl)OC(CCl)C(O)C2O)C(O)C(O)C1Cl and C=CCc1ccc(OC)cc1...done!
* Calculating Vega Similarity between #1517 :OC1COC2C(O)COC12 and COc1ccc(O)c(C(C)(C)C)c1...done!
* Calculating Vega Similarity between #1517 :OC1COC2C(O)COC12 and C=CCc1ccc(OC)cc1...done!
* Calculating Vega Similarity betw

* Calculating Vega Similarity between #1552 :CC(Oc1c(Cl)cccc1Cl)C1=NCCN1 and COc1ccc(O)c(C(C)(C)C)c1...done!
* Calculating Vega Similarity between #1552 :CC(Oc1c(Cl)cccc1Cl)C1=NCCN1 and C=CCc1ccc(OC)cc1...done!
* Calculating Vega Similarity between #1553 :O=P(OC(CCl)CCl)(OC(CCl)CCl)OC(CCl)CCl and COc1ccc(O)c(C(C)(C)C)c1...done!
* Calculating Vega Similarity between #1553 :O=P(OC(CCl)CCl)(OC(CCl)CCl)OC(CCl)CCl and C=CCc1ccc(OC)cc1...done!
* Calculating Vega Similarity between #1554 :Cn1c(N)nc2ncc(-c3ccccc3)cc21 and COc1ccc(O)c(C(C)(C)C)c1...done!
* Calculating Vega Similarity between #1554 :Cn1c(N)nc2ncc(-c3ccccc3)cc21 and C=CCc1ccc(OC)cc1...done!
* Calculating Vega Similarity between #1555 :Cc1ccc(-c2nc3ccc(C)cn3c2CC(=O)N(C)C)cc1 and COc1ccc(O)c(C(C)(C)C)c1...done!
* Calculating Vega Similarity between #1555 :Cc1ccc(-c2nc3ccc(C)cn3c2CC(=O)N(C)C)cc1 and C=CCc1ccc(OC)cc1...done!
* Calculating Vega Similarity between #1556 :Cn1c(NO)nc2ncc(-c3ccccc3)cc21 and COc1ccc(O)c(C(C)(C)C)c1...done!

* Calculating Vega Similarity between #1616 :CCC1NC(=O)C2C(Cl)C(Cl)CN2C(=O)C(CO)NC(=O)CC(c2ccccc2)NC(=O)C(CO)NC1=O and C=CCc1ccc(OC)cc1...done!
* Calculating Vega Similarity between #1617 :COc1cc(OC)c2c(c1Cl)OC1(C2=O)C(C)CC(=O)CC1OC and COc1ccc(O)c(C(C)(C)C)c1...done!
* Calculating Vega Similarity between #1617 :COc1cc(OC)c2c(c1Cl)OC1(C2=O)C(C)CC(=O)CC1OC and C=CCc1ccc(OC)cc1...done!
* Calculating Vega Similarity between #1618 :CC(=O)NC(CC(C)C)C(=O)NC(CC(C)C)C(=O)NC(C=O)CCCN=C(N)N and COc1ccc(O)c(C(C)(C)C)c1...done!
* Calculating Vega Similarity between #1618 :CC(=O)NC(CC(C)C)C(=O)NC(CC(C)C)C(=O)NC(C=O)CCCN=C(N)N and C=CCc1ccc(OC)cc1...done!
* Calculating Vega Similarity between #1619 :Clc1cccc(-c2cccc(Cl)c2)c1 and COc1ccc(O)c(C(C)(C)C)c1...done!
* Calculating Vega Similarity between #1619 :Clc1cccc(-c2cccc(Cl)c2)c1 and C=CCc1ccc(OC)cc1...done!
* Calculating Vega Similarity between #1620 :Clc1ccc(-c2ccc(Cl)c(Cl)c2Cl)c(Cl)c1Cl and COc1ccc(O)c(C(C)(C)C)c1...done!
* Calculating Vega Simil

* Calculating Vega Similarity between #1654 :CCN(Cc1cccc(S(=O)(=O)[O-])c1)c1ccc([CH2+](c2ccc(N(C)C)cc2)c2ccc(N(CC)Cc3cccc(S(=O)(=O)O)c3)cc2)cc1 and COc1ccc(O)c(C(C)(C)C)c1...done!
* Calculating Vega Similarity between #1654 :CCN(Cc1cccc(S(=O)(=O)[O-])c1)c1ccc([CH2+](c2ccc(N(C)C)cc2)c2ccc(N(CC)Cc3cccc(S(=O)(=O)O)c3)cc2)cc1 and C=CCc1ccc(OC)cc1...done!
* Calculating Vega Similarity between #1655 :CC(Cl)CCCC(Cl)CCCC(Cl)CCCC(Cl)CCCC(Cl)CCCC(Cl)CCCl and COc1ccc(O)c(C(C)(C)C)c1...done!
* Calculating Vega Similarity between #1655 :CC(Cl)CCCC(Cl)CCCC(Cl)CCCC(Cl)CCCC(Cl)CCCC(Cl)CCCl and C=CCc1ccc(OC)cc1...done!
* Calculating Vega Similarity between #1656 :Clc1cccc(-c2ccc(Cl)c(Cl)c2Cl)c1Cl and COc1ccc(O)c(C(C)(C)C)c1...done!
* Calculating Vega Similarity between #1656 :Clc1cccc(-c2ccc(Cl)c(Cl)c2Cl)c1Cl and C=CCc1ccc(OC)cc1...done!
* Calculating Vega Similarity between #1657 :Cc1ccccc1C and COc1ccc(O)c(C(C)(C)C)c1...done!
* Calculating Vega Similarity between #1657 :Cc1ccccc1C and C=CCc1ccc(OC)cc

* Calculating Vega Similarity between #1714 :CCCCCCCCCCCCCC(=O)OC1C(C)C2(O)C(C=C(CO)CC3(O)C(=O)C(C)=CC32)C2C(C)(C)C12OC(C)=O and C=CCc1ccc(OC)cc1...done!
* Calculating Vega Similarity between #1715 :Cc1cc(C)c([NH3+])cc1C and COc1ccc(O)c(C(C)(C)C)c1...done!
* Calculating Vega Similarity between #1715 :Cc1cc(C)c([NH3+])cc1C and C=CCc1ccc(OC)cc1...done!
* Calculating Vega Similarity between #1716 :Cc1c([N+](=O)[O-])c(C)c([N+](=O)[O-])c(C(C)(C)C)c1[N+](=O)[O-] and COc1ccc(O)c(C(C)(C)C)c1...done!
* Calculating Vega Similarity between #1716 :Cc1c([N+](=O)[O-])c(C)c([N+](=O)[O-])c(C(C)(C)C)c1[N+](=O)[O-] and C=CCc1ccc(OC)cc1...done!
* Calculating Vega Similarity between #1717 :ClCOCC(COCCl)OCCl and COc1ccc(O)c(C(C)(C)C)c1...done!
* Calculating Vega Similarity between #1717 :ClCOCC(COCCl)OCCl and C=CCc1ccc(OC)cc1...done!
* Calculating Vega Similarity between #1718 :COc1ccc(N)cc1 and COc1ccc(O)c(C(C)(C)C)c1...done!
* Calculating Vega Similarity between #1718 :COc1ccc(N)cc1 and C=CCc1ccc(OC)cc1.

## OUTPUT CLASS


In [17]:
obj = Generator_Molecule(df_input, df)
global tox
Tox = prevalence(Dataset)
prediction = []
lst = obj[1]
for j, i in enumerate(tqdm(lst)):
    grp = SingleMolecule.Mol_group_no_zero_removeRedundant(SingleMolecule(i.smile)).drop('SMILES', axis=1)
    if list(grp.keys()) == 0:
        # nel caso non si siano MG (ridondante con il codice di reasoning, mi seriva per velocizzare)
        prediction.append('No MG find in target')
    else:
        c1 = MoleculeInput.Calculate_grp(i)
        c = MoleculeInput.filter_simil(i)
        if len(c)<2: prediction.append('No similar substance in dataset for make a prediction')
        else:
            b = MoleculeInput.ortogonal_research(i ,Tox)
            a = MoleculeInput.reasoning(i)
            if len(a[0])==0:prediction.append(a[1])
            else: prediction.append('NON Active for exception')
                
b = pd.Series(prediction)

# non è necessario "if, else"  se il file excel di input è giusto con colonna denominata 'SMILES'
# if 'SMILES' not in df_input.keys(): df_input2 = df_input.copy().rename(columns = {0:'SMILES'})
# else:df_input2 = df_input.copy()    
# df_input2['Pred'] = b
df_input2 = obj[0].copy()
df_input2['Pred'] = b
mol_outAD_index = obj[2]
strange_molecule = pd.Series(mol_outAD_index)
if len(mol_outAD_index)>0:
    strange_molecule = pd.DataFrame(strange_molecule.transpose(), columns=['SMILES'])
    strange_molecule['SMILES_old'] = 'na'
    strange_molecule['SMILES_clean'] = 'na'
    strange_molecule['Pred'] = 'no similar in database'
    df_input2 = df_input2.append(strange_molecule)
df_input2.to_excel('prediction.xlsx')
path = os.getcwd()
os.rename('prediction.xlsx', f"{path}/result/prediction.xlsx")

100%|█████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:25<00:00, 12.78s/it]
/tmp/ipykernel_109147/2845103817.py:30: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  strange_molecule = pd.Series(mol_outAD_index)


### Class

In [13]:
# non lo vedo necessario
Dataset_Mol = []
for mol_input in (Dataset['SMILES']):
    mol2 = SingleMolecule(mol_input)
    Dataset_Mol.append(mol2)
for i, smile in enumerate(Dataset_Mol):
    if i==0:
        df_groups = SingleMolecule.Mol_group_removeRedundant(smile)
        df_alerts = BB_calculator.descriptor(Chem.MolFromSmiles(smile.Smile))
    else:
        df_groups = df_groups.append(SingleMolecule.Mol_group_removeRedundant(smile))
        df_alerts = df_alerts.append(BB_calculator.descriptor(Chem.MolFromSmiles(smile.Smile)))
df_groups = df_groups.reset_index(drop=True)
df_alerts = df_alerts.reset_index(drop=True)
df_groups = df_groups.assign(SMILES = Dataset['SMILES'])
df_alerts = df_alerts.assign(SMILES = Dataset['SMILES'], Class = Dataset['Experimental value'])
df_all = pd.merge(df_groups, df_alerts, how='inner',on='SMILES')

if len(df_all)>0 and len(set(df_all['Class']))<=2 and all(df_all['Class'].apply(lambda x: x in [0,1])):
    Carcino = df_all.loc[df_all['Class']==Dataset_class[0]]
    NoCarcino = df_all.loc[df_all['Class']==Dataset_class[1]]
else:
    Carcino = df_all.loc[df_all['Class']>-1]
    NoCarcino = df_all.loc[df_all['Class']<-2]

In [14]:
# take input object from principal class Molecule_input
std = Dataset.describe().loc[['std'],:].values[0]
mean = Dataset.describe().loc[['mean'],:].values[0]
top = list(mean+(2*std))[0]
bottom = list(mean-(2*std))[0]
limit = ((-1) - bottom) / (top - bottom)

def pdf_output_class(mol, num_mol):
    
    drop_list2 = ['TotGrp','Dissimilarity','GROUPS','Class','Similarity','SMILES',
              'Grp_Similarity_mean','Distance','Grp_Similarity_Emilio','Experimental value']
    
    pdf = FPDF(orientation='P', unit='mm', format='A4') #nuovo pdf
    Molecule_input = 'input.png'

    #for u, mol in tqdm(enumerate(lst)):

    # grp dell'input 
    grp_input = mol.target_grp_noredundant # valuta la presenza della colonna smiles?
    grp_input = grp_input[grp_input != 0].dropna(axis=1)

    if any(pd.Series(grp_input.keys()).apply(lambda x: x in ['charge-', 'charge+'])):
        if 'charge-' in grp_input.keys():
            grp_input = grp_input.drop('charge-',axis=1)
        if 'charge+' in grp_input.keys():
            grp_input = grp_input.drop('charge+',axis=1)

    #simili e gruppi optimized rdkit
    b = mol.filter_simil    
    
    if len(set(b['Experimental value']))<=2 and all(b['Experimental value'].apply(lambda x: x in [0,1])):
        output_type='Class'
    else:
        output_type='Continuo'
        
    
    # ci devono essere almeno 1 simile e 1 grp nel target 
    if len(b)>0 and len(grp_input.keys())>0:
        
        # alerts nel target rdkit
        c = mol.alerts_no_zero.drop('SMILES',axis=1)
        # orthogonal research
        SaG = mol.simili6
        b['Class'] = b['Experimental value'] # per questione di nomi di colonne
        
        if len(b)>0:
            if output_type=='Class':
                # reasoning classificazione
                if list(grp_input.keys()) == 0:
                # nel caso non si siano MG (ridondante con il codice di reasoning, mi seriva per velocizzare)
                    prediction = 'No MG find in target'
                else:
                    if len(b)<2: prediction = 'No similar substance in dataset for make a prediction'
                    else:
                        a = mol.prediction
                        if len(a[0])==0: prediction = a[1]
                        else: prediction = 'NON Active for exception'

                reas = prediction
            elif output_type=='Continuo':                                                            
                reas_intero = mol.prediction_intero

        # page pdf                        
        pdf.add_page()
        pdf.set_font("Helvetica", size = 16)
        
        # logo
        if output_type=='Continuo':
            pdf.set_xy(10.0, 19.0)
            pdf.image('PREMIER_logo.png',  link='', type='', w=74, h=20)
        else:
            pdf.set_xy(10.0, 15.0)
            pdf.image('LogConcert.png',  link='', type='', w=70, h=24)
            
        # titolo
        pdf.set_xy(70.0, 10.0)
        pdf.cell(80, 10, 'VERA: Similarity and Grouping:', 1)       
        pdf.set_font("Helvetica", size = 8)
        
        # input image
        Chem.Draw.MolToFile(Chem.MolFromSmiles(mol.smile),'input.png')
        pdf.set_xy(120.0, 30.0)
        pdf.image('input.png',  link='', type='', w=60, h=50)
        os.remove('input.png')
        
        # Mol for Chem
        mol_chem = Chem.MolFromSmiles(mol.smile)
        
        # GasteigerCharges
        AllChem.ComputeGasteigerCharges(mol_chem)
        contribs = [mol_chem.GetAtomWithIdx(i).GetDoubleProp('_GasteigerCharge') for i in range(mol_chem.GetNumAtoms())]
        fig = SimilarityMaps.GetSimilarityMapFromWeights(
            mol_chem, contribs, colorMap='jet', contourLines=10);
        fig.savefig('GasteigerCharges.png',bbox_inches='tight')
        plt.close(fig)
        
        # 'GasteigerCharges.png' position
        pdf.set_xy(90.0, 120.0)
        pdf.image('GasteigerCharges.png',  link='', type='', w=110, h=80)
        os.remove('GasteigerCharges.png')        
        pdf.text(10, 145, txt="GasteigerCharges:")
        
        # CrippenContribs
        contribs = rdMolDescriptors._CalcCrippenContribs(mol_chem)
        fig = SimilarityMaps.GetSimilarityMapFromWeights(
            mol_chem,[x for x,y in contribs], colorMap='jet', contourLines=10);
        fig.savefig('CrippenContribs.png', bbox_inches='tight')
        plt.close(fig)
        
        # 'CrippenContribs' position
        pdf.set_xy(90.0, 195.0)
        pdf.image('CrippenContribs.png',  link='', type='', w=110, h=80)
        os.remove('CrippenContribs.png')       
        pdf.text(10, 215, txt="CrippenContribs:")
        
        # Descriptors value
        pdf.set_xy(10.0, 80.0)
        des = Descriptors_Calculator(mol.smile) 
        for nd, d in enumerate(des.keys()):
            if nd == 0:               
                s = f'{d}: {round(des[d][0],3)} \n'
            else:
                s = s + f'{d}: {round(des[d][0],3)} \n'
        pdf.multi_cell(100.0, 4, txt= s) 
        # sottotili vari
        pdf_w=210
        pdf_h=297
        # molecule n and input
        pdf.text(10, (pdf_h/5)-10, txt=f"Molecule Input:")
        # Grouppi  
        pdf.text(10, (pdf_h/5)-5, txt="TotGroups: " +str(len(grp_input.keys())-1))
        pdf.set_xy(10,(pdf_h/5))
        pdf.multi_cell(100, 4, txt = str(list(grp_input.keys())[0:-1]))
        
        # alert
        if len(c.keys())>0:
            pdf.set_xy(10,(pdf_h/5)+15)
            # pdf.multi_cell(100, 4, txt = f"Structural Alers: {str(c.keys()[0])}")
            for j, string_ in enumerate(c.keys()):
                if j == 0: alert_string = str(string_)
                else: alert_string = alert_string +' '+str(string_)
                    
            pdf.multi_cell(100, 4, txt = f"Structural Alers: {alert_string}")
                              
        if b['Similarity'][0]==1:
            if output_type == 'Class':
                if b['Class'][0]==0:cl='NON-Active'
                else: cl='Active'
                pdf.set_xy(120,(pdf_h/5)+25)
                pdf.multi_cell(100, 4, txt = f"Experimental Value is known: {cl}")
            else:
                pdf.set_xy(120,(pdf_h/5)+25)
                pdf.multi_cell(100, 4, txt = f"Experimental Value is known: {b['Class'][0]} -log(mg/l)")
        
        # predizione
        pdf.set_xy(120,(pdf_h/5)+30)     
        if output_type == 'Class':          
            pdf.multi_cell(100, 4, txt = f"Prediction: {prediction}")
        else:            
            if reas_intero['reasoning'][0] == 'no': 
                txt = reas_intero['Knn'][0]
            elif reas_intero['reasoning'][0] == 'yes': 
                if reas_intero['err'][0] <= 0.5:
                    txt = reas_intero['best_value'][0]
                else:
                    txt = reas_intero['Grouping Value'][0]
            else: txt='no data'
            if type(txt)!=str:
                pdf.multi_cell(100, 4, txt = f"Prediction: {round(txt,3)} -log(mg/l)")
            else:
                pdf.multi_cell(100, 4, txt = f"Prediction: No prediction")       
        
       
        # disegni dei sei simili
        pdf.add_page()
        name_png = []
        name_barImg = []
        for j, smile in enumerate(b['SMILES']):
                if j < 6:
                    name_png.append('Simile'+str(j)+'.png')
                    if len(c.keys())>0:
                        mol_id = Chem.MolFromSmiles(smile)
                        ind = FAI.all_index_atoms(mol_id)
                        ind_bonds = FAI.all_index_bonds(mol_id)
                        img = Draw.MolToImage(mol_id, highlightAtoms=ind, highlightBonds=ind_bonds) # highlightBonds= ,highlightColor=ColorConverter().to_rgb("aqua"))
                        img.save(name_png[j])                        
                    else: Chem.Draw.MolToFile(Chem.MolFromSmiles(smile),name_png[j])
                    if output_type == 'Continuo':
                        plt.rcParams["figure.figsize"] = (3,5)
                        value = (b['Experimental value'][j] - bottom) / (top - bottom)
                        objects = (['Toxicity'])
                        x_pos = np.arange(len(objects))
                        performance = [value]
                        if value > top: value = top
                        elif value < bottom: value = bottom
                        if value<=limit: 
                            plt.bar(x_pos, performance, align='center', color='g', alpha=(1-value))
                        else: 
                            plt.bar(x_pos, performance, align='center', color='r', alpha=value)
                        plt.xticks(x_pos, objects)
                        plt.ylim(0, 1)
                        plt.axhline(y=limit)
                        plt.yticks([])
                        plt.savefig(f'bar_img{j}.png', bbox_inches='tight')
                        plt.close()
                        name_barImg.append(f'bar_img{j}.png')                        
                else: break


        # specifiche dei simili
        adj = 10
        for i in range(len(name_png)):
            if i<3:

                #Immagine
                x_imm = 115.0 
                y_imm = 30 + (90*i)

                #scritte varie
                x_write=10
                if i == 0: 
                    y_write = 10 + (90*i) + adj
                else: 
                    y_write = 10 + (90*i) + adj

                #gruppi
                x_grp=10
                if i == 0: 
                    y_grp = 40 + (90*i) + adj
                else: 
                    y_grp = 40 + (90*i) + adj

                #alerts
                x_grp_alerts = 120
                if i == 0: 
                    y_grp_alerts = 60 + (90*i) + adj
                else: 
                    y_grp_alerts = 60 + (90*i) + adj
                    
                # bar info tox intensity ( continuo values )
                if output_type == 'Continuo':
                    x_barInt = 180 
                    y_barInt = 36 + (90*i)                 

            else:
                if i == 3:
                    pdf.add_page()

                #Immagine
                x_imm = 115.0 
                y_imm = 15 + (90*(i-3))

                #scritte varie
                x_write = 10
                y_write = 10+(90*(i-3))

                #gruppi
                x_grp = 10
                y_grp = 37.8 + (90*(i-3))

                #alerts
                x_grp_alerts = 120
                y_grp_alerts = 60 + (90*(i-3))
                
                # bar info tox intensity ( continuo values )
                if output_type == 'Continuo':
                    x_barInt = 180 
                    y_barInt = 21 + (90*(i-3))   
            
            # Imagine
            pdf.set_xy(x_imm,y_imm)
            pdf.image(name_png[i],  link='', type='', w=50, h=40)                                    
            
            # bar Intensity
            if output_type == 'Continuo':
                pdf.set_xy(x_barInt, y_barInt)
                pdf.image(name_barImg[i],  link='', type='', w=15, h=35)
            
            # scritte
            # titolo pag 1
            if i == 0:
                pdf.set_font("Helvetica", size = 16) 
                pdf.set_xy(10, 10)
                pdf.cell(160, 10, f"Molecules with the highest group similarity\n", 1)
            
            # similarity etc
            pdf.set_xy(x_write, y_write)
            pdf.set_font("Helvetica", size = 8) 
            if output_type == 'Class':           
                if b['Class'][i] == 1: cl = 'Active'
                else: cl='NON-Active'
            elif output_type == 'Continuo': 
                cl = round(b['Class'][i],3)         
            if output_type == 'Class': 
                testo = f"{i+1}) SMILES: {b['SMILES'][i]}\nClass: {cl}\nSimilarityVega: {b['Similarity'][i]} \nSimilarityGrouping: {b['Grp_Similarity_Emilio'][i]} \nGrp_Similarity_Mean: {b['Grp_Similarity_mean'][i]} \nGroups in common: {b['GROUPS'][i]} \nTotGroups {b['TotGrp'][i]}\n"
            else: 
                testo = f"{i+1}) SMILES: {b['SMILES'][i]}\nExpValue: {cl} -log(mg/l)\nSimilarityVega: {b['Similarity'][i]} \nSimilarityGrouping: {b['Grp_Similarity_Emilio'][i]} \nGrp_Similarity_Mean: {b['Grp_Similarity_mean'][i]} \nGroups in common: {b['GROUPS'][i]} \nTotGroups {b['TotGrp'][i]}\n"
            pdf.multi_cell(200, 4, 
                txt = testo)
            
            # gruppi
            k = pd.DataFrame(b.iloc[i,:]).copy()
            k = k.transpose()
            k = k.drop(drop_list2, axis=1)
            k = k[k != 0].dropna(axis=1)
            for n_key, key in enumerate(k.keys()):
                if n_key == 0:
                    s1 = Tox.drop(Tox[Tox['GROUPS']!=key].index)
                    s1 = s1.reset_index()
                    s2 = s1['Tox'][0]
                    s = f"{key}   Stat: {str(s2)}\n" #Prev%: {str(s3)};\n"
                else:
                    s1 = Tox.drop(Tox[Tox['GROUPS']!=key].index)
                    s1 = s1.reset_index()
                    s2 = s1['Tox'][0]
                    s = s + f"{key}   Stat: {str(s2)}\n" #f"{key}   Stat: {str(s2)}  Prev%: {str(s3)};\n"

            #gruppi
            pdf.set_xy(x_grp, y_grp)
            pdf.multi_cell(90, 3, txt = f'GROUPS:\n{s}',border=0)

            #alerts                
            pdf.multi_cell(90, 3, txt = f'Alerts: {str(list(c))}',border=0)
        
        for j in name_png:
            os.remove(j)
        for j in name_barImg:
            os.remove(j)
            
            
        # Barchart GRP 
        # questo lo puoi fare solo se sono due classi
        
        pdf.add_page()
        # titolo
        pdf.set_font("Helvetica", size = 16) 
        pdf.set_xy(10, 10)
        pdf.cell(160, 10, "Molecule Input: Bar Chart for groups in dataset find in target", 1)
        
        pdf.set_font("Helvetica", size = 8)
        data1 = []
        data2 = []
        grp_list = list(grp_input.keys())[:-1]

        for i, grp in enumerate(grp_list):
            data1.append(Carcino[grp].sum())
            data2.append(NoCarcino[grp].sum())

        X = np.arange(len(grp_list))
        fig = plt.figure(figsize=(10,5))
        ax = fig.add_axes([0,0,1,1])
        ax.bar(X + 0.00, data1[:], color = 'r', width = 0.25)
        ax.bar(X + 0.25, data2[:], color = 'b', width = 0.25)
        ax.set_ylabel('N°_groups', size = 20)
        ax.set_title('Chemicals groups in Active-NoActive', size = 30)
        ax.set_xticks(X+0.10)
        ax.set_xticklabels(grp_list, rotation='45')
        ax.tick_params(axis='both', which='major', labelsize=20)
        ax.legend(labels=['Active', 'NoActive'])

        #figura
        x = 20
        y = 160 
        plt.savefig('fig.png',dpi=300, bbox_inches='tight') 
        pdf.set_xy(x, y)
        pdf.image('fig.png',  link='', type='', w=150, h=100)
        os.remove("fig.png")
        plt.close(fig=None)
        
        
        #Barchart Alert
        if len(list(c.keys()))>0:
            data1 = []
            data2 = []
            grp_list = list(c.keys())
            for i, grp in enumerate(grp_list):
                data1.append(Carcino[grp].sum())
                data2.append(NoCarcino[grp].sum())
            X = np.arange(len(grp_list))
            fig = plt.figure(figsize=(10,5))
            ax = fig.add_axes([0,0,1,1])
            ax.bar(X + 0.00, data1[:], color = 'r', width = 0.25)
            ax.bar(X + 0.25, data2[:], color = 'b', width = 0.25)
            ax.set_ylabel('N°_groups', size=20)
            ax.set_title('Alert in Active/NoActive', size = 30)
            ax.set_xticks(X+0.10)
            ax.set_xticklabels(grp_list, rotation='45')
            ax.tick_params(axis='both', which='major', labelsize=20)
            ax.legend(labels=['Active', 'NoActive'])

            #figura
            x = 20
            y = 30 
            plt.savefig('fig2.png', bbox_inches='tight') 
            pdf.set_xy(x, y)
            pdf.image('fig2.png',  link='', type='', w=150, h=100)
            os.remove("fig2.png")
            plt.close(fig=None)
       
        #Reasonig

        if len(SaG)>0: #and len(list(c.keys()))>1: # c'è alert 

            for i in range(len(SaG)):  
                if len(SaG[i]['SMILES'])>1:

                    pdf.add_page()
                    pdf.set_font("Arial", size = 8)
                    pdf.set_line_width(0.0)
                    pdf.rect(5.0, 5.0, 200.0,287.0)
                    pdf.rect(8.0, 8.0, 194.0,282.0)
                    
                    # titolo
                    pdf.set_font("Arial", size = 16) 
                    pdf.set_xy(10, 10)
                    pdf.cell(50, 10, "Reasoning", 1)

                    pdf.set_font("Arial", size = 8)
                    pdf.set_xy(10, 25)
                    if len(list(c.keys()))>0:
                        pdf.multi_cell(160, 4, f"Ortogonal Research based on copresence of alerts {str(list(c.keys()))} and GRP {SaG[i]['Target_Group'][0]}",
                                      1)
                    else:
                        pdf.multi_cell(160, 4, f"Ortogonal Research based on copresence of group with high prevalence {SaG[i]['Tox_reason'][0]} and GRP {SaG[i]['Target_Group'][0]}",
                                      1)

                    for z in range(len(SaG[i]['SMILES'])):

                        if z%2==0:
                            x_imm = 20 
                            y_imm = 50 + (40*(z))
                        else:
                            x_imm = 120.0 
                            y_imm = 50 + (40*(z-1)) 

                        pdf.set_xy(x_imm,y_imm)
                        
                        mol_id = Chem.MolFromSmiles(SaG[i]['SMILES'][z])
                        dic = FDT_indx.inx_groups(mol_id)
                        grp_name = SaG[i]['Target_Group'][0]
                        if type(dic[grp_name][0])!=int:
                            ind_target_grp_atoms = dic[grp_name][0][1]
                            ind_target_grp_bonds = dic[grp_name][0][0]
                        else:
                            ind_target_grp_atoms = []
                            ind_target_grp_bonds =  []
                            ind_target_grp_atoms.append(dic[grp_name][0])
                            ind_target_grp_bonds.append(dic[grp_name][0])
                            print(grp_name)
                        
                        if len(c.keys())>0:
                            
                            ind_tox_grp_atoms = FAI.all_index_atoms(mol_id)
                            ind_tox_grp_bonds = FAI.all_index_bonds(mol_id)
                            
                        else:
                            
                            grp_name = SaG[i]['Tox_reason'][0]
                            ind_tox_grp_atoms = dic[grp_name][0][1]
                            ind_tox_grp_bonds = dic[grp_name][0][0] 
                            
                        atom_cols= {}
                        for at in ind_tox_grp_atoms:
                            atom_cols[at] = (1, 0.5, 0.5)

                        bond_cols = {}
                        for bd in ind_tox_grp_bonds:
                              bond_cols[bd] = (1, 0.5, 0.5)

                        for at in ind_target_grp_atoms:
                            atom_cols[at] = (0.5, 0.6, 1)

                        for bd in ind_target_grp_bonds:
                            bond_cols[bd] = (0.5, 0.6, 1)

                        hit_ats = ind_target_grp_atoms + ind_tox_grp_atoms
                        hit_bonds = ind_target_grp_bonds + ind_tox_grp_bonds

                        drawer = rdMolDraw2D.MolDraw2DCairo(500, 500)
                        drawer.DrawMolecule(mol_id, highlightAtoms=hit_ats,
                                                 highlightAtomColors=atom_cols,
                                                 highlightBonds=hit_bonds,
                                                 highlightBondColors=bond_cols)
                        drawer.FinishDrawing()
                        with open('fig2'+str(i)+str(z)+'.png','wb') as f:
                            f.write(drawer.GetDrawingText())
                            
                        pdf.image('fig2'+str(i)+str(z)+'.png',  link='', type='', w=50, h=40)
                        pdf.set_xy(x_imm-5, y_imm-15)
                        
                        if output_type == 'Class':
                            if SaG[i]['Class'][z] == 1: cl = 'Active'
                            else: cl='NON-Active'
                        elif output_type == 'Continuo': cl = round(SaG[i]['Class'][z],3)
                        
                        if output_type == 'Class': 
                            testo = f"Smile: {SaG[i]['SMILES'][z]}\nClass: {cl}\nSimilarity: {SaG[i]['Similarity'][z]}"
                        else: 
                            testo = f"Smile: {SaG[i]['SMILES'][z]}\nExpValue: {cl} -log(mg/l)\nSimilarity: {SaG[i]['Similarity'][z]}"                         
                        
                        pdf.multi_cell(80, 4, txt = testo)
                        os.remove('fig2'+str(i)+str(z)+'.png')    
         
        # Scaffold
        
        scaffold = MoleculeInput.reasoning_on_scaffold(mol)
        scaffold.reset_index(drop=True, inplace=True)
        
        scaffold_ = scaffold_index(mol.smile)
        if len(scaffold)>0 and len(scaffold_[0])>0:
            
            pdf.add_page()
            pdf.set_font("Arial", size = 8)
            pdf.set_line_width(0.0)
            pdf.rect(5.0, 5.0, 200.0,287.0)
            pdf.rect(8.0, 8.0, 194.0,282.0)
            
            # titolo
            pdf.set_font("Arial", size = 16) 
            pdf.set_xy(10, 13)
            pdf.cell(40, 15, "Scaffold", 1)
            pdf.set_font("Arial", size = 8)
            pdf.set_xy(10, 30)          
            pdf.multi_cell(100, 4, txt = f"Scaffold analysis")
            
                          
            for z in range(len(scaffold)):
                
                scaffold_idx = scaffold_index(scaffold['SMILES'][z])
                
                if z==0: 
                    Chem.Draw.MolToFile(scaffold_idx[2],'scaffold.png')
                    x_imm = 70
                    y_imm = 30
                    pdf.set_xy(x_imm,y_imm)
                    pdf.image('scaffold.png',  link='', type='', w=70, h=60)
                    os.remove('scaffold.png')
                    
                if z<4:
                    if z%2==0:
                        x_imm = 20 
                        y_imm = 50 + (40*(z+2))
                    else:
                        x_imm = 120.0 
                        y_imm = 50 + (40*(z+1)) 

                    pdf.set_xy(x_imm,y_imm)
                    
                    
                    # Chem.Draw.MolToFile(Chem.MolFromSmiles(scaffold['SMILES'][z]),'fig2'+str(z)+'.png')

                    img = Draw.MolToImage(Chem.MolFromSmiles(scaffold['SMILES'][z]), 
                                          highlightAtoms=scaffold_idx[1],
                                          highlightBonds=scaffold_idx[0],
                                          highlightColor=ColorConverter().to_rgb('aqua')
                                          ) 

                    img.save('fig2'+str(z)+'.png') 
                    pdf.image('fig2'+str(z)+'.png',  link='', type='', w=50, h=40)

                    pdf.set_xy(x_imm-5, y_imm-15)
                    if output_type == 'Class':
                        if scaffold['Class'][z] == 1:cl = 'Active'
                        else: cl='NON-Active'
                    elif output_type == 'Continuo':
                        cl = round(scaffold['Class'][z],3)


                    if output_type == 'Class': 
                        testo = f"Smile: {scaffold['SMILES'][z]}\nClass: {cl}\nSimilarity: {scaffold['Similarity'][z]}"
                    else: 
                        testo = f"Smile: {scaffold['SMILES'][z]}\nExpValue: {cl} -log(mg/l)\nSimilarity: {scaffold['Similarity'][z]}"                        

                    pdf.multi_cell(80, 4, txt = testo)
                    os.remove('fig2'+str(z)+'.png')
                    
                    
                    
        # descriptor similarity
        
        desc_similarity = MoleculeInput.descriptor_similarity(mol)
           
        if len(desc_similarity)>0:
            
            pdf.add_page()
            pdf.set_font("Arial", size = 8)
            pdf.set_line_width(0.0)
            pdf.rect(5.0, 5.0, 200.0,287.0)
            pdf.rect(8.0, 8.0, 194.0,282.0)
            
            # titolo
            pdf.set_xy(12, 13)
            pdf.set_font("Arial", size = 16)             
            pdf.cell(40, 15, "Descriptors", 1)
            pdf.set_font("Arial", size = 8)
            pdf.set_xy(12, 30)
            pdf.multi_cell(100, 4, txt = f"Calculator Similarity descriptor")
            
                          
            for j, z in enumerate(range(len(desc_similarity))):
                
                if j<6:
                    if z%2==0:
                        x_imm = 20 
                        y_imm = 60 + (40*(z))
                    else:
                        x_imm = 120.0 
                        y_imm = 60 + (40*(z-1)) 

                    pdf.set_xy(x_imm,y_imm)
                    Chem.Draw.MolToFile(Chem.MolFromSmiles(desc_similarity['SMILES'][z]),'figdescr'+str(z)+'.png')

                    pdf.image('figdescr'+str(z)+'.png',  link='', type='', w=40, h=30)
                    pdf.set_xy(x_imm-5, y_imm-25)
                    if output_type == 'Class':
                        if desc_similarity['Class'][z] == 1:cl = 'Active'
                        else: cl='NON-Active'
                    elif output_type == 'Continuo':
                        cl = round(desc_similarity['Class'][z],3)
                    
                    
                    if output_type == 'Class': 
                        testo = f"Smile: {desc_similarity['SMILES'][z]}\nClass: {cl}\nDistanceCityBlock: {round(desc_similarity['descriptor_similarity'][z],3)}\nLogP:{desc_similarity['LogP'][z]}\nMW: {desc_similarity['ExactMolWt'][z]}"
                    else: 
                        testo = f"Smile: {desc_similarity['SMILES'][z]}\nExpValue: {cl} -log(mg/l)\nDistanceCityBlock: {round(desc_similarity['descriptor_similarity'][z],3)}\nLogP:{desc_similarity['LogP'][z]}\nMW: {desc_similarity['ExactMolWt'][z]}"                      
                        
                    pdf.multi_cell(80, 4, txt = testo)
                    os.remove('figdescr'+str(z)+'.png')

        pdf.output(f"Mol{num_mol}.pdf")                                                   

In [16]:
for i in range(len(lst)):
    if i<5:
        pdf_output_class(lst[i],i)
        path = os.getcwd()
        os.rename(f"Mol{i}.pdf", f"{path}/result/Mol{i}.pdf")
    